In [51]:
%pip install -q -U beautifulsoup4
%pip install -q -U google-generativeai

In [52]:
stock_name = 'AAPL'
news_url = f'https://finance.yahoo.com/quote/{stock_name}/news/'
statistics_url = f'https://finance.yahoo.com/quote/{stock_name}/key-statistics/'

In [53]:
import requests
from bs4 import BeautifulSoup

headers = {
    "Referer": f"https://finance.yahoo.com/quote/{stock_name}/",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
}

news_response = requests.get(news_url, headers=headers)
news_soup = BeautifulSoup(news_response.text, "html.parser")

In [54]:
news_title = news_soup.select(".story-item h3")
news_desc = news_soup.select(".story-item p")
news_related = news_soup.select(".story-item .yf-13tszyc")

news_data = []
for title, desc, rel in zip(news_title, news_desc, news_related):
    news_data.append([{
        'title': title.text.strip(),
        'desc': desc.text.strip(),
        "related": rel.text.replace('  ', '').replace('% ', ', ')
    }])

news_data

[[{'title': 'Learn how to use Apple Intelligence’s ‘Image Playground’',
   'desc': "Apple just released iOS 18.1.1, but it's iOS 18.2 that will come with exciting image generation features, such as “Image Playground,” Apple's image creation feature that creates cartoon-like images from text descriptions. Here’s what you need to know about the Apple Intelligence feature, including what it is, how to use it, and when it launches. What is Image Playground? Image Playground is part of Apple Intelligence, a range of AI-driven tools from Apple. The new feature enables users to enter",
   'related': 'AAPL +0.95%'}],
 [{'title': 'Indonesia expects $1 billion investment commitment from Apple in a\xa0week',
   'desc': 'Indonesia is expecting to get a $1 billion investment commitment from tech firm Apple Inc in a week, its investment minister said on Tuesday, after the government banned iPhone 16 sales for failing to meet local content rules.  Indonesia stopped sales of the smartphone because it 

In [55]:
statistics_response = requests.get(statistics_url, headers=headers)
statistics_soup = BeautifulSoup(statistics_response.text, "html.parser")

In [56]:
import pandas as pd

valuation_table_th = statistics_soup.select("section.yf-kbx2lo table tr th")
valuation_table_td = statistics_soup.select("section.yf-kbx2lo table tr td")

v_table = []

v_table.append({"Session": [th.text for th in valuation_table_th[1:]]})

current_key = ""
for idx, td in enumerate(valuation_table_td):
    if idx == 0 or idx % 7 == 0:
        v_table.append({td.text: []})
        current_key = td.text
    else:
        v_table[-1][current_key].append(td.text)


valuation_string = ""
for val in v_table:
    print(list(val.keys())[0], "|", ", ".join(val[list(val.keys())[0]]))
    valuation_string += (
        f"{list(val.keys())[0]} | {', '.join(val[list(val.keys())[0]])}\n"
    )

Session | Current, 9/30/2024, 6/30/2024, 3/31/2024, 12/31/2023, 9/30/2023, 9/30/2023 - 12/12/1980
Market Cap | 3.62T , 3.52T , 3.23T , 2.65T , 2.99T , 2.68T 
Enterprise Value | 3.66T , 3.56T , 3.27T , 2.68T , 3.06T , 2.72T 
Trailing P/E | 39.41 , 35.46 , 32.76 , 26.67 , 31.41 , 28.73 
Forward P/E | 32.15 , 30.67 , 28.65 , 26.32 , 29.15 , 25.77 
PEG Ratio (5yr expected) | 2.48 , 2.24 , 2.23 , 2.11 , 2.31 , 2.18 
Price/Sales | 9.44 , 9.38 , 8.62 , 6.99 , 7.94 , 7.10 
Price/Book | 63.59 , 52.80 , 43.21 , 35.49 , 47.90 , 44.17 
Enterprise Value/Revenue | 9.37 , 9.24 , 8.56 , 6.96 , 7.98 , 7.09 
Enterprise Value/EBITDA | 27.20 , 26.35 , 24.57 , 20.10 , 23.66 , 21.46 
Upgrade to begin using 40 years of earnings data and get so much more.  Build financial models with decades of earnings stats, ratios, and valuation data — all exportable — to power your trade ideas. Upgrade   | 


In [57]:
trading = statistics_soup.select("section.yf-14j5zka table tr td")

trading_string = ""
trading_info = {}
for idx, t in enumerate(trading):
    if idx % 2 == 0:
        k = t.text.strip().replace('  ', ' ')
        if k[-1] in '0123456789':
            k = k[:-1].strip()
        trading_info[k] = trading[idx + 1].text.strip()
        trading_string += f"{k} | {trading[idx + 1].text.strip()}\n"
        print(k, "|", trading[idx + 1].text.strip())

Fiscal Year Ends | 9/28/2024
Most Recent Quarter (mrq) | 9/28/2024
Profit Margin | 23.97%
Operating Margin (ttm) | 31.17%
Return on Assets (ttm) | 21.46%
Return on Equity (ttm) | 157.41%
Revenue (ttm) | 391.03B
Revenue Per Share (ttm) | 25.49
Quarterly Revenue Growth (yoy) | 6.10%
Gross Profit (ttm) | --
EBITDA | 134.66B
Net Income Avi to Common (ttm) | 93.74B
Diluted EPS (ttm) | 6.09
Quarterly Earnings Growth (yoy) | -35.80%
Total Cash (mrq) | 65.17B
Total Cash Per Share (mrq) | 4.31
Total Debt (mrq) | 119.06B
Total Debt/Equity (mrq) | 209.06%
Current Ratio (mrq) | 0.87
Book Value Per Share (mrq) | 3.77
Operating Cash Flow (ttm) | 118.25B
Levered Free Cash Flow (ttm) | 110.85B
Beta (5Y Monthly) | 1.24
52 Week Range | 23.87%
S&P 500 52-Week Change | 32.40%
52 Week High | 240.79
52 Week Low | 164.08
50-Day Moving Average | 229.11
200-Day Moving Average | 206.11
Avg Vol (3 month) | 48.36M
Avg Vol (10 day) | 43.69M
Shares Outstanding | 15.12B
Implied Shares Outstanding | 15.33B
Float | 15

In [58]:
from google.colab import userdata

import google.generativeai as genai

genai.configure(api_key=userdata.get('GEMINI_API_KEY'))

In [59]:
import typing_extensions as typing


class Analyse(typing.TypedDict):
    analyse: list[str]
    suggestion: list[str]
    buy: bool


generation_config = {
    "temperature": 0,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_schema": Analyse,
    "response_mime_type": "application/json",
}

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
    system_instruction="你是專業的股市分析師，曾經利用過去的新聞與統計資料進行分析。分析時，必須提出完整資訊，包括正面與負面，並使用繁體中文回答。",
)

chat_session = model.start_chat(
    history=[
        {
            "role": "user",
            "parts": [
                "以下是最新的股市新聞：",
                str(news_data),
                "\n\n",
                "以下是最新的股市統計資料：",
                valuation_string,
                "\n\n",
                "以下是最新的股市交易資料：",
                trading_string,
                "\n\n",
            ],
        }
    ]
)

In [60]:
import json
from IPython.display import display, Markdown

response = chat_session.send_message("請分析並提出建議")
response_json = json.loads(response.text)

In [61]:
for ana in response_json['analyse']:
    display(Markdown(ana))
print("\n====================\n")

for sug in response_json['suggestion']:
    display(Markdown(sug))
print("\n====================\n")

print(f"建議買進：{response_json['buy']}")


根據提供的新聞和數據，蘋果公司(AAPL)的股價表現強勁，並受到多位知名投資者看好，預期2025年表現良好。然而，也存在一些潛在的風險，例如來自中國市場的競爭加劇、員工提出的隱私和言論相關訴訟，以及印尼市場因當地內容規定而暫停銷售iPhone 16等。

**正面因素：**
* 多位知名投資者(例如Ken Fisher, Louis Navellier)將AAPL列入其2025年投資組合，顯示市場對其長期增長潛力充滿信心。
* 新的iOS 18系統加入AI相關功能，例如Image Playground，提升產品競爭力。
* 儘管中國市場存在挑戰，但蘋果在其他市場的表現依然強勁，iPhone 16的需求依然存在。
* 財務數據顯示，蘋果公司擁有強勁的盈利能力和現金流，以及相對穩定的股價表現。
* Apple Pay與Coinbase合作，拓展支付領域。

**負面因素：**
* 蘋果公司面臨員工提出的訴訟，指控其監控員工個人隱私和限制言論自由，這可能會對公司聲譽造成負面影響，並產生法律費用。
* 印尼市場因當地內容規定而暫停銷售iPhone 16，顯示蘋果公司在遵守不同國家/地區的法規方面面臨挑戰。
* 中國市場競爭加劇，可能影響蘋果公司的銷售額和市場份額。
* 儘管季度營收增長，但季度盈利增長卻大幅下降，值得關注。
* 高負債比率(Total Debt/Equity (mrq) | 209.06%)值得關注。

**綜合分析：**
蘋果公司目前處於一個相對樂觀的狀態，但潛在的風險不容忽視。其強勁的財務狀況和長期增長潛力使其成為一個有吸引力的投資標的，但投資者也需要密切關注其在中國市場的表現、員工訴訟的進展以及其他潛在的風險。

建議投資者在投資AAPL前，應仔細評估其正面和負面因素，並根據自身的風險承受能力做出投資決策。可以考慮分批建倉，降低投資風險。同時，密切關注公司新聞和財務數據，以便及時調整投資策略。



建議買進：True
